In [ ]:
import pandas as pd
import numpy as np

In [ ]:
population_data_to_load = "raw_data/population.csv"
number_house_data_to_load = "raw_data/number_houing_units_state.xlsx"
value_housing_data_to_load = "raw_data/single_family_housing_value_zillow.csv"

# Read population and housing Data File and store into Pandas DataFrames
population_data = pd.read_csv(population_data_to_load)
number_house_data = pd.read_excel(number_house_data_to_load)
zillow_df = pd.read_csv(value_housing_data_to_load)

In [ ]:
population_data.head(20)

In [ ]:
population_data.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE'], axis=1, inplace=True)
#population_data.head()

In [ ]:
population_data.columns

In [ ]:
population_data.count()

In [ ]:
no_null_population_data = population_data.dropna(how='any')

In [ ]:
no_null_population_data.count()

In [ ]:
no_null_population_data["NAME"].value_counts()

In [ ]:
# net migration per state
net_state_migration = population_data[["NAME","POPESTIMATE2010","POPESTIMATE2011","POPESTIMATE2012"
                                       ,"POPESTIMATE2013","POPESTIMATE2014","POPESTIMATE2015"
                                       ,"POPESTIMATE2016","POPESTIMATE2017","POPESTIMATE2018"
                                       ,"POPESTIMATE2019", "NETMIG2010", "NETMIG2011", "NETMIG2012", 
                                   "NETMIG2013", "NETMIG2014", "NETMIG2015", 
                                  "NETMIG2016", "NETMIG2017","NETMIG2018","NETMIG2019"]]
net_state_migration.head(20)

In [ ]:
net_state_migration.drop([0, 1, 2, 3, 4, 56], inplace=True)

In [ ]:
net_state_migration.reset_index(drop=True)

In [ ]:
state_migrate = net_state_migration.rename(columns = { "NAME" : "State" })
state_migrate

In [ ]:
#number of house units
number_house_data.head(70)

In [ ]:
# Delete unwanted rows
number_house_data.drop([0,1,2,3,4,5,6,7,59,60,61,62,63], axis=0, inplace=True)

In [ ]:
# Delete unwanted columns
number_house_data.drop(number_house_data.iloc[:, 1:3], axis=1, inplace=True)
number_house_data

In [ ]:
# Rename to format for later use 
number_house_data.rename(columns={'table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)' : 'State',
                                 'Unnamed: 3' : 'HUESTIMATE2010','Unnamed: 4' : 'HUESTIMATE2011','Unnamed: 5' : 'HUESTIMATE2012','Unnamed: 6' : 'HUESTIMATE2013',
                                 'Unnamed: 7' : 'HUESTIMATE2014','Unnamed: 8' : 'HUESTIMATE2015','Unnamed: 9' : 'HUESTIMATE2016','Unnamed: 10' : 'HUESTIMATE2017',
                                 'Unnamed: 11' : 'HUESTIMATE2018','Unnamed: 12' : 'HUESTIMATE2019',}, inplace=True)
number_house_data.head()

In [ ]:
# Removing the leading '.' in the state names
number_house_data['State'] = number_house_data['State'].str[1:]

In [ ]:
number_house_data.head()

In [ ]:
number_house_data.reset_index(drop=True)

In [ ]:
zillow_df.head()

In [ ]:
# We want to use State for final merge
zillow_df.rename(columns={'RegionName' : 'State'}, inplace=True)

In [ ]:
# Remove columns near State so that we have only State followed by the monthly data
zillow_df.drop(columns=['RegionID', 'SizeRank', 'RegionType', 'StateName'], inplace=True)

In [ ]:
# Create variable to store drop index list
years_to_drop = []

In [ ]:
# Create a list of index values for columns we want to remove:  Start at 1 to skip State then start after 10 years 12*10 =120
# Keep going until it reaches the end of the dataframe
for i in range(1,len(zillow_df.columns)):
    if (i < 121)|(i > 240): #keeping the second group of 12 * 10 months
        years_to_drop.append(i)

In [ ]:
#Delete all the unwanted columns from the list
zillow_df.drop(zillow_df.columns[years_to_drop], axis=1, inplace=True)

In [ ]:
# See how many empty or nan cells there are
zillow_df.isnull().sum().sum()

In [ ]:
#Create a new dataframe to hold the annual average housing cost starting with State
annual_average_cost_df = pd.DataFrame().assign(State = zillow_df['State'])
annual_average_cost_df

In [ ]:
# Populate new dataframe with the row average over a year using nanmean to average around the invalid cells.  Using indexes to avoid the column names.
ys = 0
for year in range(10,20):
    annual_average_cost_df[f'20{year} Mean Housing'] =  np.nanmean(zillow_df.iloc[:,[1 + ys,2 + ys,3 + ys,4 + ys,5 + ys,6 + ys,7 + ys,8 + ys,9 + ys,10 + ys,11 + ys,12 + ys]], axis=1)
    ys += 12

In [ ]:
annual_average_cost_df

In [ ]:
net_state_migration

In [ ]:
merged_data_frame = pd.merge(state_migrate, number_house_data, how='left', on=['State','State'] ) 

In [ ]:
merged_data_frame

In [ ]:
merged_data = pd.merge(merged_data_frame, annual_average_cost_df, how='left', on=['State','State'] ) 
merged_data

In [ ]:
merged_data.to_csv("Resources/final_data.csv")